In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# Quick Start

In [ ]:
# Construct S.A.M. instance
sam = holo.sam.Semi_Analytic_Model(mtot=[2.75e5*MSOL, 1.0e11*MSOL, 23], mrat=[0.02, 1.0, 25], redz=[0.0, 6.0, 31])

In [ ]:
# Choose observed GW frequencies
fobs = utils.nyquist_freqs(15.0*YR, 0.1*YR)
print(utils.stats(1/(fobs*YR)), "[yr]")

In [ ]:
# Sample SAM into discrete binaries
# individual binaries have `weights` = 1.0,
# representative centroids of high density bins have `weights` > 1
# `dens` and `edges` are the actual density of binaries and the corresponding grid-edges
vals, weights, edges, dens = holo.sam.sample_sam_with_hardening(
    sam, holo.evolution.Hard_GW, fobs=fobs,
    sample_threshold=10.0, cut_below_mass=1e7, limit_merger_time=4*GYR,
)

In [ ]:
gwf_freqs, gwf, gwb = holo.sam._gws_from_samples(vals, weights, fobs)

fig, ax = plot.figax()

xx = kale.utils.midpoints(fobs) * YR   # [1/sec] ==> [1/yr]

amp = 10e-16
yy = amp * np.power(xx, -2/3)
ax.plot(xx, yy, 'k--', alpha=0.25)

ax.plot(xx, gwb, 'k-')

idx = (gwf > gwb)
xx = gwf_freqs * YR   # [1/sec] ==> [1/yr]
ax.scatter(xx[idx], gwf[idx], color='r', s=20, alpha=0.5)
ax.scatter(xx[~idx], gwf[~idx], edgecolor='r', facecolor='none', s=20, alpha=0.5)

# ax.plot(fobs*YR, gwb_smooth, 'b--')
# ax.plot(fobs*YR, np.median(gwb_rough, axis=-1), 'b:')
# ax.fill_between(fobs*YR, *np.percentile(gwb_rough, [25, 75], axis=-1), color='b', alpha=0.25)

plt.show()